In [4]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import warnings
import time
warnings.filterwarnings(action='ignore')

In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
import time
import csv

# Set up the Chrome driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# URL of the website
url = 'https://www.thecocktailproject.com/search-recipes/'

# Open the website
driver.get(url)
driver.maximize_window()

# Wait for the page to load
time.sleep(1)

# Find the birthday input fields and enter the date
month_input = driver.find_element(By.ID, 'edit-month-of-birth')
month_input.send_keys('08')

day_input = driver.find_element(By.ID, 'edit-day-of-birth')
day_input.send_keys('20')

year_input = driver.find_element(By.ID, 'year')
year_input.send_keys('1999')
time.sleep(1)

app_submit = driver.find_element(By.CSS_SELECTOR, 'app-submit')
button_within_submit = app_submit.find_element(By.CSS_SELECTOR, 'button#edit-submit')
button_within_submit.click()
time.sleep(1)

reject_all_button = driver.find_element(By.ID, 'onetrust-reject-all-handler')
reject_all_button.click()
time.sleep(1)

#### 여기까지 access

# Find all cocktail elements
# Initialize an empty dictionary to store cocktail names and links
cocktails_dict = {}

# Loop through each page from 1 to 22
for i in range(0, 22):
    # Get the URL of the page
    page_url = f"https://www.thecocktailproject.com/search-recipes/?page={i}"
    # Visit the page
    driver.get(page_url)
    
    # Find all cocktail elements
    cocktail_elements = driver.find_elements(By.CSS_SELECTOR, '.receipe-grid-items')

    # Loop through each cocktail element on the current page
    for cocktail_element in cocktail_elements:
    # Extract name of the cocktail
        name = cocktail_element.find_element(By.CSS_SELECTOR, '.item-desc a').text.strip()
        
        # Extract link of the cocktail
        link = cocktail_element.find_element(By.CSS_SELECTOR, '.item-detail a').get_attribute('href')
        
        driver.get(link)

        ingredient_elements = driver.find_elements(By.CSS_SELECTOR, '.ingredients-block')
        ingredients = [ingredient.text.strip() for ingredient in ingredient_elements]

        spirit_elements = driver.find_elements(By.CLASS_NAME, 'drink-properties-incredible')
        for spirit in spirit_elements:
            try:
                spirits = spirit_elements[0].find_element(By.TAG_NAME,'p').find_element(By.TAG_NAME,'a').text.strip() 
            except NoSuchElementException:
                spirits='N/A'
            try:
                flavors = spirit_elements[1].find_element(By.TAG_NAME,'p').find_element(By.TAG_NAME,'a').text.strip()
            except NoSuchElementException:
                flavors='N/A'
                
        rating_elements = driver.find_elements(By.CSS_SELECTOR, '.rate-fivestar-btn-filled')
        ratings = len(rating_elements)

        rating_count_elements = driver.find_elements(By.CSS_SELECTOR, '.item-list ul li')
        ratings_count = [rating_count.text.strip() for rating_count in rating_count_elements]
        try:
            ratings_count = [int(count) for count in ratings_count if count.isdigit()][0]
        except IndexError:
            ratings_count = 0


    # Add the name and link of the cocktail to the dictionary
        cocktails_dict[name] = {'link': link, 'ingredients': ingredients, 'main_spirit': spirits, 'flavor': flavors, 'rating': ratings, 'ratings_count': ratings_count}

        driver.back()

    
        
driver.quit()




In [20]:
import csv
import re

csv_file = 'cocktails.csv'

# Define fieldnames based on the keys of the dictionary
fieldnames = ['Name', 'Link', 'Ingredients', 'Main Spirit', 'Flavor', 'Rating', 'Ratings Count']

# Open the CSV file in 'write' mode
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    # Create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # Write header
    writer.writeheader()

    # Write data rows
    for name, cocktail_data in cocktails_dict.items():

        cleaned_name = re.sub(r'[^\w\s]', '', name)

        writer.writerow({'Name': cleaned_name,
                         'Link': cocktail_data['link'],
                         'Ingredients': '\n'.join(cocktail_data['ingredients']),
                         'Main Spirit': cocktail_data['main_spirit'],
                         'Flavor': cocktail_data['flavor'],
                         'Rating': cocktail_data['rating'],
                         'Ratings Count': cocktail_data['ratings_count']})

PermissionError: [Errno 13] Permission denied: 'cocktails.csv'

In [8]:
len(cocktails_dict)

440

In [7]:
cocktails_dict['Lazy Day Colada']

{'link': 'https://www.thecocktailproject.com/drink-recipes/lazy-day-colada',
 'ingredients': ['INGREDIENTS\n1 part\nDeKuyper® Pucker® Sour Apple Schnapps\n1 part\nCruzan® Aged Light Rum\n2 1/4 parts\nPina Colada Mix\nSHOPPING LIST BUY NOW'],
 'main_spirit': 'Cordials',
 'flavor': 'Sweet',
 'rating': 3,
 'ratings_count': 40}

In [18]:
import pandas as pd
df1 = pd.read_csv('cocktails_data.csv')
df2 = pd.read_csv('cocktails.csv')
merged_df = pd.merge(df1, df2, on=['Name', 'link, 'ingredients'], how='outer')
merged_df.to_csv('merged_file.csv', index=False) 

KeyError: 'name'